In [10]:
import pyreadr
transactions_df = pyreadr.read_r("/home/spadef/data/IDEA_DeFi_Research/Data/Lending_Protocols/Aave/V3/Polygon/transactionsFinSurvival.rds")[None]
# transactions_df.to_csv("/home/spadef/data/IDEA_DeFi_Research/Data/Lending_Protocols/Aave/V3/Polygon/transactionsFinSurvival.csv", index=False)
print(transactions_df["timestamp"].max())

1755726959.0


In [2]:
import os
import pandas as pd

EVENTS = ["Deposit", "Withdraw", "Repay", "Borrow", "Liquidated"]
DATA_PATH = "./data/"

In [ ]:
users = {}
for index_event in EVENTS:
    for outcome_event in EVENTS:
        csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
        if not os.path.exists(csv_path):
            continue
        df = pd.read_csv(csv_path)
        for _, row in df.iterrows():
            if row["user"] not in users:
                users[row["user"]] = {
                    "user_address": "0xUser123",
                    "description": "My custom user profile",
                    "transactions": [],
                }
            users[row["user"]]["transactions"].append(
                {"action": row[""], "symbol": "USDC", "amount": 5000, "timestamp": 0}
            )

In [4]:
for index_event in EVENTS:
    for outcome_event in EVENTS:
        if not (index_event == "Deposit" and outcome_event == "Withdraw"):
            continue
        csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
        if not os.path.exists(csv_path):
            continue
        df = pd.read_csv(csv_path)
        for col in ["Index Event", "Outcome Event"]:
            if col in df.columns:
                mask = df[col].notna()
                df.loc[mask, col] = (
                    df.loc[mask, col]
                    .astype(str)
                    .str.replace("account liquidated", "liquidated", case=False, regex=False)
                    .str.strip()
                )
        df.to_csv(csv_path, index=False)
        print(f"Updated {csv_path}")

Updated ./data/Deposit/Withdraw/data.csv


In [5]:
for index_event in EVENTS:
    for outcome_event in EVENTS:
        if not (index_event == "Deposit" and outcome_event == "Withdraw"):
            continue
        csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
        if not os.path.exists(csv_path):
            continue
        df = pd.read_csv(csv_path)
        for col in ["Index Event", "Outcome Event"]:
            if col in df.columns:
                mask = df[col].notna()
                df.loc[mask, col] = (
                    df.loc[mask, col]
                    .astype(str)
                    .str.replace("liquidation", "liquidated", case=False, regex=False)
                    .str.strip()
                )
        df.to_csv(csv_path, index=False)
        print(f"Updated {csv_path}")

Updated ./data/Deposit/Withdraw/data.csv


In [ ]:
for event in EVENTS:
    csv_path = os.path.join(DATA_PATH, event, event, "data.csv")
    if not (event == "Deposit" and event == "Withdraw"):
        continue
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    for col in ["Outcome Event"]:
        if col in df.columns:
            mask = df[col].notna()
            df.loc[mask, col] = (
                df.loc[mask, col]
                .astype(str)
                .str.replace(event + "_outcome", event, case=False, regex=False)
                .str.strip()
            )
    df.to_csv(csv_path, index=False)
    print(f"Updated {csv_path}")

In [9]:
for event in EVENTS:
    csv_path = os.path.join(DATA_PATH, event, event, "data.csv")
    if not (event == "Deposit" and event == "Withdraw"):
        continue
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    for col in ["Outcome Event"]:
        if col in df.columns:
            mask = df[col].notna()
            df.loc[mask, col] = (
                df.loc[mask, col]
                .astype(str)
                .str.replace(event, event.lower(), case=False, regex=False)
                .str.strip()
            )
    df.to_csv(csv_path, index=False)
    print(f"Updated {csv_path}")

In [ ]:
index_event = "Liquidated"
for outcome_event in EVENTS:
    csv_path = os.path.join(DATA_PATH, index_event, outcome_event, "data.csv")
    if not os.path.exists(csv_path):
        continue
    df = pd.read_csv(csv_path)
    for col in ["Index Event", "Outcome Event"]:
        if col in df.columns:
            mask = df[col].notna()
            df.loc[mask, col] = (
                df.loc[mask, col]
                .astype(str)
                .str.replace("liquidation", "liquidated", case=False, regex=False)
                .str.strip()
            )
    df.to_csv(csv_path, index=False)
    print(f"Updated {csv_path}")

Updated ./data/Liquidated/Deposit/data.csv
Updated ./data/Liquidated/Withdraw/data.csv
Updated ./data/Liquidated/Repay/data.csv
Updated ./data/Liquidated/Borrow/data.csv
Updated ./data/Liquidated/Liquidated/data.csv


In [17]:
import json
from collections import defaultdict

usersToTest = [
    "0xae0cc724fc85cd394435eb4f9ddfd3a8c088b4e2",
    "0x0e7b3aea0428137786edf20617670179435c530b",
    "0x97dc47d17ab56c32a4d2da1839e3ff261304e95d",
    "0x5091d57b4feb57db1da04b51cc5454f371b623a6",
]

for user in usersToTest:
    with open(
        f"../data/IDEA_DeFi_Research/Data/CSV/profile-generation/user_{user}.json"
    ) as f:
        userTest = json.load(f)

    userTransactions = userTest["transactions"]
    userTransactions = userTransactions[
        : next(
            (i for i, e in enumerate(userTransactions) if (e["action"] == "Liquidated")), -1
        )
    ]
    borrows = defaultdict(int)
    repays = defaultdict(int)
    for transaction in userTransactions:
        if transaction["action"] == "Borrow":
            borrows[transaction["symbol"]] += transaction["amount"]
        elif transaction["action"] == "Repay":
            repays[transaction["symbol"]] += transaction["amount"]
    print(f'{user}\'s"Debt" before liquidation: {[f'{symbol}: {borrows[symbol] - repays[symbol]}' for symbol in borrows.keys()]}')

0xae0cc724fc85cd394435eb4f9ddfd3a8c088b4e2's"Debt" before liquidation: ['USDC: 898.7291915312042', 'WETH: -1.6900000000003718e-06']
0x0e7b3aea0428137786edf20617670179435c530b's"Debt" before liquidation: ['USDC: 0.0', 'WETH: 75.7625662202381']
0x97dc47d17ab56c32a4d2da1839e3ff261304e95d's"Debt" before liquidation: ['USDC: 7.854323']
0x5091d57b4feb57db1da04b51cc5454f371b623a6's"Debt" before liquidation: ['USDC: 0.0']
